#Speech Emotion Recognition with MLP Classifier



# Mount google drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Install following libraries

In [ ]:
!pip install librosa soundfile numpy sklearn pyaudio

  ERROR: Failed building wheel for pyaudio
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-a7g8z_ru/pyaudio_0c7d38b29d21456d98230cfa206c654f/setup.py'"'"'; __file__='"'"'/tmp/pip-install-a7g8z_ru/pyaudio_0c7d38b29d21456d98230cfa206c654f/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-n3mcs352/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/pyaudio Check the logs for full command output.


In [ ]:
!pip install soundfile

# Make the necessary imports

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

* mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
* chroma: Pertains to the 12 different pitch classes
* mel: Mel Spectrogram Frequency

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS & TESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral','calm','happy','sad','angry','fearful', 'disgust','surprised']

# Load the data and extract features for each sound file

In [ ]:

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob('/content/drive/My Drive/Audio_Song_Actors_01-24/Actor_*/*.wav'):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75,random_state=9)

In [ ]:
# Split the dataset
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

#Observe the shape of the training and testing datasets:

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(4439, 1480)


# Number of features extracted.

In [ ]:
# Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


# MLP Classifier

In [ ]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#Fit/train the model.

In [ ]:
# Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

# Predict the accuracy of our model

In [ ]:
# Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# Print the accuracy    
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 80.81%


#classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

       angry       0.94      0.91      0.93       223
        calm       0.85      0.81      0.83       238
     disgust       0.50      0.93      0.65       108
     fearful       0.93      0.85      0.89       240
       happy       0.70      0.93      0.80       214
     neutral       0.77      0.71      0.74       108
         sad       0.96      0.64      0.77       246
   surprised       0.91      0.60      0.73       103

    accuracy                           0.81      1480
   macro avg       0.82      0.80      0.79      1480
weighted avg       0.85      0.81      0.81      1480



# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,y_pred)
print (matrix)

# THANK YOU